# CPU graph

A simple example of running several CPU intensive nodes in parallel and aggregating the results.

## Example worker

The example worker code is [here](./example_workers/cpu_worker/main.py).
It uses an encryption function to simulate a CPU intensive activity.

Since this worker uses the Tierkreis Python library,
we can automatically generate stub files using the following command.
The stub files will provide us with type hints 

In [10]:
# Generate stubs file.
!cd ./example_workers/cpu_worker && uv run main.py --stubs-path ../../cpu_worker_stubs.py
# Format and lint generated file.
!uv run ruff format cpu_worker_stubs.py
!uv run ruff check --fix cpu_worker_stubs.py

1 file reformatted
Found 1 error (1 fixed, 0 remaining).


## Example graph

We can import this stub file to help create our graph.

In [18]:
from typing import NamedTuple
from tierkreis.controller.data.core import EmptyModel, TKRRef
from tierkreis.controller.data.graph import GraphBuilder
from cpu_worker_stubs import encrypt, encryptOutput
from tests.tkr_builtins import mean

def map_body() -> GraphBuilder[TKRRef[str], encryptOutput]:
    g = GraphBuilder(TKRRef[str])
    result = g.fn(encrypt(plaintext=g.inputs, work_factor=g.const(2048)))
    return g.outputs(result)

class GraphOutputs(NamedTuple):
    average_time_taken: TKRRef[float]
    ciphertexts: TKRRef[list[bytes]]

def graph() -> GraphBuilder[EmptyModel, GraphOutputs]:
    g = GraphBuilder()
    body = g.graph_const(map_body())
    plaintexts_list = g.const([f"plaintext+{n}" for n in range(20)])
    plaintexts = g.unfold_list(plaintexts_list)
    results = g.map(body, plaintexts)

    ciphertexts = results.map(lambda x: x.ciphertext)
    ciphertexts_list = g.fold_list(ciphertexts)

    times = results.map(lambda x: x.time_taken)
    times_list = g.fold_list(times)

    av = g.fn(mean(values=times_list))
    out = GraphOutputs(ciphertexts=ciphertexts_list, average_time_taken=av)

    return g.outputs(out)


ImportError: cannot import name 'mean' from 'tests.tkr_builtins' (/Users/matthew.burke/Documents/GitHub/tierkreis/tierkreis/tests/tkr_builtins.py)

In [11]:
from pathlib import Path
from uuid import UUID
from tierkreis.controller import run_graph
from tierkreis.controller.executor.uv_executor import UvExecutor
from tierkreis.controller.storage.filestorage import ControllerFileStorage

storage = ControllerFileStorage(UUID(int=2048), "cpu_graph", do_cleanup=True)
executor = UvExecutor(registry_path=Path("./example_workers"),logs_path=storage.logs_path)
run_graph(storage, executor, graph().data, {})

NameError: name 'graph' is not defined